In [1]:
import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''

In [2]:
import tensorflow as tf
import numpy as np
import json
from glob import glob

2022-05-30 21:20:02.496117: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.11.0


In [3]:
phonemes = glob('*-phoneme.json')
phonemes

['kelantan-phoneme.json',
 'johor-phoneme.json',
 'melaka-phoneme.json',
 'penang-phoneme.json',
 'pahang-phoneme.json',
 'kedah-phoneme.json',
 'negeri-sembilan-phoneme.json']

In [4]:
import re

replace_chars = '.,!?['
left, right = [], []
for f in phonemes:
    with open(f) as fopen:
        data = json.load(fopen)
    for k, v in data.items():
        l = k
        for c in replace_chars:
            l = l.replace(c, f' ')
        l = l.lower()
        l = re.sub(r'[ ]+', ' ', l).strip()
        r = re.sub(r'[ ]+', ' ', v.replace(']', ' ')).strip()
        if len(l) and len(r):
            left.append(l)
            right.append(r)

In [5]:
PAD = 0
EOS = 1
UNK = 2
GO = 3
[PAD, EOS, UNK, GO]

[0, 1, 2, 3]

In [6]:
left_dict = [PAD, EOS, UNK, GO] + sorted(set(list(''.join(left))))
left_dict = {c: no for no, c in enumerate(left_dict)}

In [32]:
right_dict = [PAD, EOS, UNK, GO] + sorted(set(list(''.join(right))))
right_dict = {c: no for no, c in enumerate(right_dict)}
rev_right_dict = {v: k for k, v in right_dict.items()}

In [8]:
left_dict

{0: 0,
 1: 1,
 2: 2,
 3: 3,
 ' ': 4,
 "'": 5,
 '-': 6,
 '`': 7,
 'a': 8,
 'b': 9,
 'c': 10,
 'd': 11,
 'e': 12,
 'f': 13,
 'g': 14,
 'h': 15,
 'i': 16,
 'j': 17,
 'k': 18,
 'l': 19,
 'm': 20,
 'n': 21,
 'o': 22,
 'p': 23,
 'q': 24,
 'r': 25,
 's': 26,
 't': 27,
 'u': 28,
 'w': 29,
 'y': 30,
 'z': 31}

In [94]:
right_dict

{0: 0,
 1: 1,
 2: 2,
 3: 3,
 ' ': 4,
 ')': 5,
 ',': 6,
 '.': 7,
 '/': 8,
 ':': 9,
 'A': 10,
 'C': 11,
 'E': 12,
 'I': 13,
 'N': 14,
 'S': 15,
 'Z': 16,
 'a': 17,
 'b': 18,
 'd': 19,
 'e': 20,
 'f': 21,
 'g': 22,
 'h': 23,
 'i': 24,
 'j': 25,
 'k': 26,
 'l': 27,
 'm': 28,
 'n': 29,
 'o': 30,
 'p': 31,
 'r': 32,
 's': 33,
 't': 34,
 'u': 35,
 'w': 36,
 'z': 37,
 '\x8d': 38,
 '«': 39,
 '\xad': 40,
 'Ä': 41,
 'Ò': 42,
 'Ö': 43,
 'â': 44,
 'ø': 45,
 'ù': 46}

In [9]:
class Translator:
    def __init__(self, size_layer, num_layers, embedded_size,
                 from_dict_size, to_dict_size, learning_rate, beam_width = 10):
        
        def cells(reuse=False):
            return tf.nn.rnn_cell.LSTMCell(size_layer,initializer=tf.orthogonal_initializer(),reuse=reuse)
        
        def attention(encoder_out, seq_len, reuse=False):
            attention_mechanism = tf.contrib.seq2seq.BahdanauAttention(num_units = size_layer, 
                                                                    memory = encoder_out,
                                                                    memory_sequence_length = seq_len)
            return tf.contrib.seq2seq.AttentionWrapper(
            cell = tf.nn.rnn_cell.MultiRNNCell([cells(reuse) for _ in range(num_layers)]), 
                attention_mechanism = attention_mechanism,
                attention_layer_size = size_layer)
        
        self.X = tf.placeholder(tf.int32, [None, None])
        self.Y = tf.placeholder(tf.int32, [None, None])
        self.X_seq_len = tf.count_nonzero(self.X, 1, dtype=tf.int32)
        self.Y_seq_len = tf.count_nonzero(self.Y, 1, dtype=tf.int32)
        batch_size = tf.shape(self.X)[0]
        
        encoder_embedding = tf.Variable(tf.random_uniform([from_dict_size, embedded_size], -1, 1))
        decoder_embedding = tf.Variable(tf.random_uniform([to_dict_size, embedded_size], -1, 1))
        
        encoder_out, encoder_state = tf.nn.dynamic_rnn(
            cell = tf.nn.rnn_cell.MultiRNNCell([cells() for _ in range(num_layers)]), 
            inputs = tf.nn.embedding_lookup(encoder_embedding, self.X),
            sequence_length = self.X_seq_len,
            dtype = tf.float32)
        main = tf.strided_slice(self.Y, [0, 0], [batch_size, -1], [1, 1])
        decoder_input = tf.concat([tf.fill([batch_size, 1], GO), main], 1)
        dense = tf.layers.Dense(to_dict_size)
        
        with tf.variable_scope('decode'):
            decoder_cells = attention(encoder_out, self.X_seq_len)
            training_helper = tf.contrib.seq2seq.TrainingHelper(
                inputs = tf.nn.embedding_lookup(decoder_embedding, decoder_input),
                sequence_length = self.Y_seq_len,
                time_major = False)
            training_decoder = tf.contrib.seq2seq.BasicDecoder(
                    cell = decoder_cells,
                    helper = training_helper,
                    initial_state = decoder_cells.zero_state(batch_size, tf.float32).clone(cell_state=encoder_state),
                    output_layer = dense)
            training_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                    decoder = training_decoder,
                    impute_finished = True,
                    maximum_iterations = tf.reduce_max(self.Y_seq_len))
            self.training_logits = training_decoder_output.rnn_output
            
        with tf.variable_scope('decode', reuse=True):
            predicting_helper = tf.contrib.seq2seq.GreedyEmbeddingHelper(
                embedding = decoder_embedding,
                start_tokens = tf.tile(tf.constant([GO], dtype=tf.int32), [batch_size]),
                end_token = EOS)
            predicting_decoder = tf.contrib.seq2seq.BasicDecoder(
                    cell = decoder_cells,
                    helper = predicting_helper,
                    initial_state = decoder_cells.zero_state(batch_size, tf.float32).clone(cell_state=encoder_state),
                    output_layer = dense)
            predicting_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                    decoder = predicting_decoder,
                    impute_finished = True,
                    maximum_iterations = 2 * tf.reduce_max(self.X_seq_len))
            self.greedy = predicting_decoder_output.sample_id
            self.greedy = tf.identity(self.greedy,name='greedy')
        
        with tf.variable_scope('decode', reuse=True):
            
            encoder_out_tiled = tf.contrib.seq2seq.tile_batch(encoder_out, beam_width)
            encoder_state_tiled = tf.contrib.seq2seq.tile_batch(encoder_state, beam_width)
            X_seq_len_tiled = tf.contrib.seq2seq.tile_batch(self.X_seq_len, beam_width)
            decoder_cell = attention(encoder_out_tiled, X_seq_len_tiled, reuse=True)
            
            predicting_decoder = tf.contrib.seq2seq.BeamSearchDecoder(
                cell = decoder_cell,
                embedding = decoder_embedding,
                start_tokens = tf.tile(tf.constant([GO], dtype=tf.int32), [batch_size]),
                end_token = EOS,
                initial_state = decoder_cell.zero_state(batch_size * beam_width, tf.float32).clone(
                    cell_state = encoder_state_tiled),
                beam_width = beam_width,
                output_layer = dense,
                length_penalty_weight = 0.0)
            
            predicting_decoder_output, _, _ = tf.contrib.seq2seq.dynamic_decode(
                decoder = predicting_decoder,
                impute_finished = False,
                maximum_iterations = tf.reduce_max(self.X_seq_len))
            
            self.beam = predicting_decoder_output.predicted_ids[:, :, 0]
            self.beam = tf.identity(self.beam,name='beam')
        
        masks = tf.sequence_mask(self.Y_seq_len, tf.reduce_max(self.Y_seq_len), dtype=tf.float32)
        self.masks = masks
        self.cost = tf.contrib.seq2seq.sequence_loss(logits = self.training_logits,
                                                     targets = self.Y,
                                                     weights = masks)
        self.optimizer = tf.train.AdamOptimizer(learning_rate = learning_rate).minimize(self.cost)
        y_t = tf.argmax(self.training_logits,axis=2)
        y_t = tf.cast(y_t, tf.int32)
        self.prediction = tf.boolean_mask(y_t, masks)
        mask_label = tf.boolean_mask(self.Y, masks)
        correct_pred = tf.equal(self.prediction, mask_label)
        correct_index = tf.cast(correct_pred, tf.float32)
        self.accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [10]:
size_layer = 256
num_layers = 2
embedded_size = 256
learning_rate = 1e-3
batch_size = 32
epoch = 20

In [11]:
tf.reset_default_graph()
sess = tf.InteractiveSession()
model = Translator(size_layer, num_layers, embedded_size, len(left_dict), len(right_dict), learning_rate)
sess.run(tf.global_variables_initializer())







The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



2022-05-30 21:20:03.026855: I tensorflow/core/platform/profile_utils/cpu_utils.cc:109] CPU Frequency: 2496000000 Hz
2022-05-30 21:20:03.027345: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x3cd3d40 initialized for platform Host (this does not guarantee that XLA will be used). Devices:
2022-05-30 21:20:03.027358: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): Host, Default Version
2022-05-30 21:20:03.028390: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2022-05-30 21:20:03.030529: E tensorflow/stream_executor/cuda/cuda_driver.cc:282] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-05-30 21:20:03.030545: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: huseincomel-desktop
2022-05-30 21:20:03.030549: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: huseincomel

In [12]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'lstm/model.ckpt')

'lstm/model.ckpt'

In [13]:
def pad_sentence_batch(sentence_batch, pad_int):
    padded_seqs = []
    seq_lens = []
    max_sentence_len = max([len(sentence) for sentence in sentence_batch])
    for sentence in sentence_batch:
        padded_seqs.append(sentence + [pad_int] * (max_sentence_len - len(sentence)))
        seq_lens.append(len(sentence))
    return padded_seqs, seq_lens

In [14]:
train_X, train_Y = [], []
for i in range(len(left)):
    train_X.append([left_dict[c] for c in left[i]] + [1])
    train_Y.append([right_dict[c] for c in right[i]] + [1])

In [15]:
batch_x, _ = pad_sentence_batch(train_X[: 5], 0)
batch_y, _ = pad_sentence_batch(train_Y[: 5], 0)

In [16]:
np.array(batch_y).shape

(5, 14)

In [17]:
np.array(batch_x).shape

(5, 11)

In [18]:
%%time

sess.run([model.cost, model.optimizer], feed_dict = {model.X: batch_x, model.Y: batch_y})

CPU times: user 417 ms, sys: 103 ms, total: 520 ms
Wall time: 250 ms


[3.8444717, None]

In [20]:
greedy, beam = sess.run([model.greedy, model.beam], feed_dict = {model.X: train_X[:1]})

In [21]:
greedy, beam

(array([[7, 1]], dtype=int32), array([[1, 1, 1, 1, 1]], dtype=int32))

In [23]:
epoch, batch_size

(20, 32)

In [68]:
import tqdm
from sklearn.utils import shuffle

for e in range(2):
    train_X, train_Y = shuffle(train_X, train_Y)
    pbar = tqdm.tqdm(
        range(0, len(train_X), batch_size), desc = 'minibatch loop')
    train_loss, train_acc = [], []
    for i in pbar:
        index = min(i + batch_size, len(train_X))
        batch_x, seq_x = pad_sentence_batch(train_X[i : index], PAD)
        batch_y, seq_y = pad_sentence_batch(train_Y[i : index], PAD)
        feed = {model.X: batch_x,
                model.Y: batch_y}
        accuracy, loss, _ = sess.run([model.accuracy,model.cost,model.optimizer],
                                    feed_dict = feed)
        train_loss.append(loss)
        train_acc.append(accuracy)
        pbar.set_postfix(cost = loss, accuracy = accuracy)
    
    print('epoch %d, training avg loss %f, training avg acc %f'%(e+1,
                                                                 np.mean(train_loss),np.mean(train_acc)))

minibatch loop: 100%|██████████| 19/19 [00:09<00:00,  2.05it/s, accuracy=1, cost=0.00721]    


epoch 1, training avg loss 0.009723, training avg acc 0.998853


minibatch loop: 100%|██████████| 19/19 [00:09<00:00,  1.95it/s, accuracy=1, cost=0.0048]     

epoch 2, training avg loss 0.006597, training avg acc 0.999433


In [69]:
saver = tf.train.Saver(tf.trainable_variables())
saver.save(sess, 'lstm-bahdanau/model.ckpt')

'lstm-bahdanau/model.ckpt'

In [74]:
string = 'comel'
batch = [left_dict[c] for c in string] + [1]

In [75]:
greedy, beam = sess.run([model.greedy, model.beam], feed_dict = {model.X: [batch]})

In [76]:
greedy, beam

(array([[34, 15, 38, 28,  7, 15, 39,  7, 27, 10, 28,  1]], dtype=int32),
 array([[34, 15, 38, 28,  7, 15]], dtype=int32))

In [77]:
''.join([rev_right_dict[i] for i in greedy[0] if i > 3])

'tS\x8dm.S«.lAm'

In [78]:
strings = ','.join(
    [
        n.name
        for n in tf.get_default_graph().as_graph_def().node
        if ('Variable' in n.op
        or 'Placeholder' in n.name
        or 'greedy' in n.name
        or 'beam' in n.name
        or 'alphas' in n.name)
        and 'Adam' not in n.name
        and 'beta' not in n.name
        and 'OptimizeLoss' not in n.name
        and 'Global_Step' not in n.name
    ]
)
strings.split(',')

['Placeholder',
 'Placeholder_1',
 'Variable',
 'Variable_1',
 'rnn/multi_rnn_cell/cell_0/lstm_cell/kernel',
 'rnn/multi_rnn_cell/cell_0/lstm_cell/bias',
 'rnn/multi_rnn_cell/cell_1/lstm_cell/kernel',
 'rnn/multi_rnn_cell/cell_1/lstm_cell/bias',
 'decode/memory_layer/kernel',
 'decode/decoder/attention_wrapper/multi_rnn_cell/cell_0/lstm_cell/kernel',
 'decode/decoder/attention_wrapper/multi_rnn_cell/cell_0/lstm_cell/bias',
 'decode/decoder/attention_wrapper/multi_rnn_cell/cell_1/lstm_cell/kernel',
 'decode/decoder/attention_wrapper/multi_rnn_cell/cell_1/lstm_cell/bias',
 'decode/decoder/attention_wrapper/bahdanau_attention/query_layer/kernel',
 'decode/decoder/attention_wrapper/bahdanau_attention/attention_v',
 'decode/decoder/attention_wrapper/attention_layer/kernel',
 'decode/decoder/dense/kernel',
 'decode/decoder/dense/bias',
 'decode_1/greedy',
 'decode_2/decoder/while/BeamSearchDecoderStep/beam_width',
 'decode_2/decoder/while/BeamSearchDecoderStep/next_beam_probs/range/start',
 

In [79]:
def freeze_graph(model_dir, output_node_names):

    if not tf.gfile.Exists(model_dir):
        raise AssertionError(
            "Export directory doesn't exists. Please specify an export "
            "directory: %s" % model_dir)

    checkpoint = tf.train.get_checkpoint_state(model_dir)
    input_checkpoint = checkpoint.model_checkpoint_path
    
    absolute_model_dir = "/".join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_dir + "/frozen_model.pb"
    clear_devices = True
    with tf.Session(graph=tf.Graph()) as sess:
        saver = tf.train.import_meta_graph(input_checkpoint + '.meta', clear_devices=clear_devices)
        saver.restore(sess, input_checkpoint)
        output_graph_def = tf.graph_util.convert_variables_to_constants(
            sess,
            tf.get_default_graph().as_graph_def(),
            output_node_names.split(",")
        ) 
        with tf.gfile.GFile(output_graph, "wb") as f:
            f.write(output_graph_def.SerializeToString())
        print("%d ops in the final graph." % len(output_graph_def.node))

In [80]:
freeze_graph("lstm-bahdanau", strings)




INFO:tensorflow:Restoring parameters from lstm-bahdanau/model.ckpt
INFO:tensorflow:Froze 16 variables.
INFO:tensorflow:Converted 16 variables to const ops.

1649 ops in the final graph.


In [81]:
def load_graph(frozen_graph_filename):
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())
    with tf.Graph().as_default() as graph:
        tf.import_graph_def(graph_def)
    return graph

In [82]:
g=load_graph('lstm-bahdanau/frozen_model.pb')

In [83]:
x = g.get_tensor_by_name('import/Placeholder:0')
i_greedy = g.get_tensor_by_name('import/decode_1/greedy:0')
i_beam = g.get_tensor_by_name('import/decode_2/beam:0')

In [84]:
test_sess = tf.InteractiveSession(graph=g)

/home/ubuntu/tf-nvidia/lib/python3.8/site-packages/tensorflow_core/python/client/session.py:1750: UserWarning: An interactive session is already active. This can cause out-of-memory errors in some cases. You must explicitly call `InteractiveSession.close()` to release resources held by the other session(s).
  warnings.warn('An interactive session is already active. This can '


In [85]:
greedy, beam = test_sess.run([i_greedy, i_beam], feed_dict = {x: [batch]})

In [86]:
''.join([rev_right_dict[i] for i in greedy[0] if i > 3])

'tS\x8dm.S«.lAm'

In [87]:
from tensorflow.tools.graph_transforms import TransformGraph
from tensorflow.contrib.seq2seq.python.ops import beam_search_ops

In [88]:
transforms = ['add_default_attributes',
             'remove_nodes(op=Identity, op=CheckNumerics, op=Dropout)',
             'fold_batch_norms',
             'fold_old_batch_norms',
             'quantize_weights(fallback_min=-10, fallback_max=10)',
             'strip_unused_nodes',
             'sort_by_execution_order']

pb = 'lstm-bahdanau/frozen_model.pb'
input_graph_def = tf.GraphDef()
with tf.gfile.FastGFile(pb, 'rb') as f:
    input_graph_def.ParseFromString(f.read())

transformed_graph_def = TransformGraph(input_graph_def, 
                                       ['Placeholder'],
                                       ['decode_1/greedy', 'decode_2/beam'], transforms)

with tf.gfile.GFile(f'{pb}.quantized', 'wb') as f:
    f.write(transformed_graph_def.SerializeToString())

2022-05-30 21:36:49.682102: I tensorflow/tools/graph_transforms/transform_graph.cc:318] Applying add_default_attributes
2022-05-30 21:36:49.687661: I tensorflow/tools/graph_transforms/transform_graph.cc:318] Applying remove_nodes
2022-05-30 21:36:49.693479: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for decode_1/greedy
2022-05-30 21:36:49.694247: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for decode_2/beam
2022-05-30 21:36:49.701659: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for decode_1/greedy
2022-05-30 21:36:49.702217: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for decode_2/beam
2022-05-30 21:36:49.708453: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for decode_1/greedy
2022-05-30 21:36:49.709027: I tensorflow/tools/graph_transforms/remove_nodes.cc:78] Skipping replacement for decode_2/beam
2022-05-30 21:36:49.728517

In [90]:
!tar -cvf phoneme-lstm-bahdanau.tar lstm-bahdanau

lstm-bahdanau/
lstm-bahdanau/checkpoint
lstm-bahdanau/frozen_model.pb.quantized
lstm-bahdanau/model.ckpt.index
lstm-bahdanau/model.ckpt.data-00000-of-00001
lstm-bahdanau/model.ckpt.meta
lstm-bahdanau/frozen_model.pb


In [89]:
from malaya_boilerplate.huggingface import upload_dict

/home/ubuntu/tf-nvidia/lib/python3.8/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [91]:
files_mapping = {'lstm-bahdanau/frozen_model.pb': 'model.pb'}
upload_dict(model = 'phoneme-lstm-bahdanau', files_mapping = files_mapping)

/home/ubuntu/tf-nvidia/lib/python3.8/site-packages/huggingface_hub/hf_api.py:79: FutureWarning: `name` and `organization` input arguments are deprecated and will be removed in v0.8. Pass `repo_id` instead.
  warnings.warn(


In [92]:
files_mapping = {'lstm-bahdanau/frozen_model.pb.quantized': 'model.pb'}
upload_dict(model = 'phoneme-lstm-bahdanau-quantized', files_mapping = files_mapping)

In [93]:
files_mapping = {'phoneme-lstm-bahdanau.tar': 'phoneme-lstm-bahdanau.tar'}
upload_dict(model = 'pretrained-phoneme', files_mapping = files_mapping)